In [ ]:
import sys
sys.path.append('./src')
import warnings
warnings.filterwarnings("ignore")

from data import PPCI

In [ ]:
# load dataset
encoder = "dino"
dataset = PPCI(encoder = encoder,
               token = "class",
               task = "or",
               split_criteria = "position_easy",
               environment = "supervised",
               batch_size = 64,
               num_proc = 4,
               verbose = True,
               data_dir = 'data/istant_hq',
               results_dir = f'results/istant_hq/{encoder}')

In [ ]:
# example train
dataset.train(add_pred_env="supervised", 
            hidden_layers = 1,
            hidden_nodes = 256,
            batch_size = 128,
            lr = 0.0005,
            num_epochs=5,
            save = False,
            verbose=True,
            multidomain=True,
            ic_weight=0,
            seed=1)
dataset.evaluate(color=None, verbose=False)